# Agentic RAG with Autogen

ဒီ notebook က Autogen agents တွေကို အသုံးပြုပြီး Retrieval-Augmented Generation (RAG) ကို အကဲဖြတ်နိုင်စွမ်း မြှင့်တင်ထားတဲ့ နည်းလမ်းနဲ့ အကောင်အထည်ဖော်ပြသပေးမှာ ဖြစ်ပါတယ်။


SQLite ဗားရှင်းပြင်ဆင်ခြင်း  
သင်အောက်ပါအမှားကိုတွေ့ရှိပါက -  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```
  
သင်၏ notebook ရဲ့အစမှာ ဒီ code block ကို uncomment လုပ်ပါ:


In [2]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [ ]:
!pip install chromadb

In [3]:
import os
import time
import asyncio
from typing import List, Dict
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
from azure.core.credentials import AzureKeyCredential
from autogen_ext.models.azure import AzureAIChatCompletionClient

import chromadb

load_dotenv()

True

## Client ကိုဖန်တီးပါ

ပထမဦးဆုံး၊ Azure AI Chat Completion Client ကို initialize လုပ်ပါ။ ဒီ client ကို အသုံးပြုပြီး Azure OpenAI service နှင့် ဆက်သွယ်ကာ အသုံးပြုသူရဲ့ မေးခွန်းများအတွက် အဖြေများကို ဖန်တီးရန် အသုံးပြုမည်ဖြစ်သည်။


In [4]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

## ဗက်တာဒေတာဘေစ်စ် စတင်ခြင်း

ChromaDB ကို အမြဲတမ်းသိုလှောင်မှုနှင့်အတူ စတင်ပြီး အဆင့်မြှင့်ထားသော နမူနာစာရွက်များကို ထည့်သွင်းပါသည်။ ChromaDB ကို တိကျသော အဖြေများကို ဖန်တီးရန် အကြောင်းအရာပေးသော စာရွက်များကို သိုလှောင်ရန်နှင့် ပြန်လည်ရယူရန် အသုံးပြုမည်ဖြစ်သည်။


In [5]:
# Initialize ChromaDB with persistent storage
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.create_collection(
    name="travel_documents",
    metadata={"description": "travel_service"},
    get_or_create=True
)

# Enhanced sample documents
documents = [
    "Contoso Travel offers luxury vacation packages to exotic destinations worldwide.",
    "Our premium travel services include personalized itinerary planning and 24/7 concierge support.",
    "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.",
    "Popular destinations include the Maldives, Swiss Alps, and African safaris.",
    "Contoso Travel provides exclusive access to boutique hotels and private guided tours."
]

# Add documents with metadata
collection.add(
    documents=documents,
    ids=[f"doc_{i}" for i in range(len(documents))],
    metadatas=[{"source": "training", "type": "explanation"} for _ in documents]
)

## Context Provider အကောင်အထည်ဖော်ခြင်း

`ContextProvider` class သည် အကြောင်းအရာများကို အမျိုးမျိုးသောရင်းမြစ်များမှ ရယူခြင်းနှင့် ပေါင်းစည်းခြင်းကို စီမံခန့်ခွဲသည်။

1. **Vector Database ရယူခြင်း**: ChromaDB ကို အသုံးပြု၍ ခရီးသွားစာရွက်စာတမ်းများအပေါ် semantic ရှာဖွေမှုကို ဆောင်ရွက်သည်။
2. **ရာသီဥတုအချက်အလက်**: မြို့ကြီးများအတွက် simulation လုပ်ထားသော ရာသီဥတု database ကို ထိန်းသိမ်းထားသည်။
3. **ပေါင်းစည်းထားသော အကြောင်းအရာ**: စာရွက်စာတမ်းနှင့် ရာသီဥတုအချက်အလက်များကို ပေါင်းစည်းပြီး အပြည့်အစုံသော အကြောင်းအရာကို ဖန်တီးသည်။

အဓိက method များ:
- `get_retrieval_context()`: query အပေါ်မူတည်၍ သက်ဆိုင်ရာ စာရွက်စာတမ်းများကို ရယူသည်။
- `get_weather_data()`: သတ်မှတ်ထားသောနေရာများအတွက် ရာသီဥတုအချက်အလက်ကို ပေးသည်။
- `get_unified_context()`: စာရွက်စာတမ်းနှင့် ရာသီဥတုအကြောင်းအရာများကို ပေါင်းစည်းပြီး တိုးတက်သော အဖြေများအတွက် အသုံးပြုသည်။


In [6]:
class ContextProvider:
    def __init__(self, collection):
        self.collection = collection
        # Simulated weather database
        self.weather_database = {
            "new york": {"temperature": 72, "condition": "Partly Cloudy", "humidity": 65, "wind": "10 mph"},
            "london": {"temperature": 60, "condition": "Rainy", "humidity": 80, "wind": "15 mph"},
            "tokyo": {"temperature": 75, "condition": "Sunny", "humidity": 50, "wind": "5 mph"},
            "sydney": {"temperature": 80, "condition": "Clear", "humidity": 45, "wind": "12 mph"},
            "paris": {"temperature": 68, "condition": "Cloudy", "humidity": 70, "wind": "8 mph"},
        }
    
    def get_retrieval_context(self, query: str) -> str:
        """Retrieves relevant documents from vector database based on query."""
        results = self.collection.query(
            query_texts=[query],
            include=["documents", "metadatas"],
            n_results=2
        )
        context_strings = []
        if results and results.get("documents") and len(results["documents"][0]) > 0:
            for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
                context_strings.append(f"Document: {doc}\nMetadata: {meta}")
        return "\n\n".join(context_strings) if context_strings else "No relevant documents found"
    
    def get_weather_data(self, location: str) -> str:
        """Simulates retrieving weather data for a given location."""
        if not location:
            return ""
            
        location_key = location.lower()
        if location_key in self.weather_database:
            data = self.weather_database[location_key]
            return f"Weather for {location.title()}:\n" \
                   f"Temperature: {data['temperature']}°F\n" \
                   f"Condition: {data['condition']}\n" \
                   f"Humidity: {data['humidity']}%\n" \
                   f"Wind: {data['wind']}"
        else:
            return f"No weather data available for {location}."
    
    def get_unified_context(self, query: str, location: str = None) -> str:
        """Returns a unified context combining both document retrieval and weather data."""
        retrieval_context = self.get_retrieval_context(query)
        
        weather_context = ""
        if location:
            weather_context = self.get_weather_data(location)
            weather_intro = f"\nWeather Information for {location}:\n"
        else:
            weather_intro = ""
        
        return f"Retrieved Context:\n{retrieval_context}\n\n{weather_intro}{weather_context}"

## Agent Configuration

ကျွန်ုပ်တို့သည် retrieval agent နှင့် assistant agent များကို ပြင်ဆင်ထားပါသည်။ Retrieval agent သည် semantic search ကို အသုံးပြု၍ သက်ဆိုင်သော အချက်အလက်များကို ရှာဖွေရာတွင် အထူးကျွမ်းကျင်ပြီး၊ assistant သည် ရှာဖွေထားသော အချက်အလက်များအပေါ် အခြေခံ၍ အသေးစိတ် အဖြေများကို ဖန်တီးပေးပါသည်။


In [7]:
# Create agents with enhanced capabilities
assistant = AssistantAgent(
    name="assistant",
    model_client=client,
    system_message=(
        "You are a helpful AI assistant that provides answers using ONLY the provided context. "
        "Do NOT include any external information. Base your answer entirely on the context given below."
    ),
)

## RAG ဖြင့် Query ကို အလုပ်လုပ်စေခြင်း

ကျွန်ုပ်တို့သည် `ask_rag` function ကို သတ်မှတ်ပြီး query ကို assistant သို့ ပို့ပေးကာ response ကို အလုပ်လုပ်စေပြီး အကဲဖြတ်သည်။ ဒီ function သည် assistant နှင့် အပြန်အလှန် ဆက်သွယ်မှုကို စီမံခန့်ခွဲပြီး response ၏ အရည်အသွေးကို တိုင်းတာရန် evaluator ကို အသုံးပြုသည်။


In [8]:
async def ask_rag_agent(query: str, context_provider: ContextProvider, location: str = None):
    """
    Sends a query to the assistant agent with context from the provider.
    
    Args:
        query: The user's question
        context_provider: The context provider instance
        location: Optional location for weather queries
    """
    try:
        # Get unified context
        context = context_provider.get_unified_context(query, location)
        
        # Augment the query with context
        augmented_query = (
            f"{context}\n\n"
            f"User Query: {query}\n\n"
            "Based ONLY on the above context, please provide a helpful answer."
        )

        # Send the augmented query to the assistant
        start_time = time.time()
        response = await assistant.on_messages(
            [TextMessage(content=augmented_query, source="user")],
            cancellation_token=CancellationToken(),
        )
        processing_time = time.time() - start_time
        
        return {
            'query': query,
            'response': response.chat_message.content,
            'processing_time': processing_time,
            'location': location
        }
    except Exception as e:
        print(f"Error processing query: {e}")
        return None

# အသုံးပြုမှု ဥပမာ

ကျွန်တော်တို့ evaluator ကို initialize လုပ်ပြီး ကျွန်တော်တို့ process လုပ်ပြီး အကဲဖြတ်လိုသော queries များကို သတ်မှတ်ပါသည်။


In [ ]:
async def main():
    # Initialize context provider
    context_provider = ContextProvider(collection)
    
    # Example queries
    queries = [
        {"query": "What does Contoso's travel insurance cover?"},
        {"query": "What's the weather like in London?", "location": "london"},
        {"query": "What luxury destinations does Contoso offer and what's the weather in Paris?", "location": "paris"},
    ]
    
    print("=== Autogen RAG Demo ===")
    for query_data in queries:
        query = query_data["query"]
        location = query_data.get("location")
        
        print(f"\n\nQuery: {query}")
        if location:
            print(f"Location: {location}")
        
        # Show the context being used
        context = context_provider.get_unified_context(query, location)
        print("\n--- Context Used ---")
        print(context)
        print("-------------------")
        
        # Get response from the agent
        result = await ask_rag_agent(query, context_provider, location)
        if result:
            print(f"\nResponse: {result['response']}")
        print("\n" + "="*50)

## စက်ရုပ်ကို အလုပ်လုပ်စေပါ

ကျွန်တော်တို့ စက်ရုပ်ကို အပြန်အလှန် အလုပ်လုပ်နိုင်တဲ့ ပတ်ဝန်းကျင်မှာလား၊ သို့မဟုတ် သာမန် စက်ရုပ်လားဆိုတာ စစ်ဆေးပြီး အဓိကလုပ်ဆောင်မှုကို သင့်တော်အောင် အလုပ်လုပ်စေပါ။


In [12]:
if __name__ == "__main__":
    if asyncio.get_event_loop().is_running():
        await main()
    else:
        asyncio.run(main())

=== Autogen RAG Demo ===


Query: What does Contoso's travel insurance cover?

--- Context Used ---
Retrieved Context:
Document: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.
Metadata: {'source': 'training', 'type': 'explanation'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'source': 'training', 'type': 'explanation'}


-------------------

Response: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.



Query: What's the weather like in London?
Location: london

--- Context Used ---
Retrieved Context:
Document: Popular destinations include the Maldives, Swiss Alps, and African safaris.
Metadata: {'source': 'training', 'type': 'explanation'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'source': 'training', 'type': 'explanation'}


Weather Information for london:
Weather for London:
T


---

**ဝန်ခံချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှန်ကန်မှုအတွက် ကြိုးစားနေသော်လည်း၊ အလိုအလျောက်ဘာသာပြန်ဆိုမှုများတွင် အမှားများ သို့မဟုတ် မမှန်ကန်မှုများ ပါဝင်နိုင်သည်ကို သတိပြုပါ။ မူလဘာသာစကားဖြင့် ရေးသားထားသော စာရွက်စာတမ်းကို အာဏာတည်သော ရင်းမြစ်အဖြစ် သတ်မှတ်သင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူ့ဘာသာပြန်ပညာရှင်များမှ ဘာသာပြန်ဆိုမှုကို အကြံပြုပါသည်။ ဤဘာသာပြန်ကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲသုံးစားမှု သို့မဟုတ် အနားလွဲမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
